In [2]:
from os import getcwd
import os.path as op
from glob import glob

In [3]:
from clients.utils import create_allocation_table
from clients.expt_recipes.db_query import create_plates_from_allocation_table

expt_name = 'A81_E217'
allocation_table = create_allocation_table(expt_name)
expt_plates = create_plates_from_allocation_table(allocation_table)

In [4]:
from hardware.qpcr import QpcrDataFile

id_qpcr_plate = 'A81_E217_1'
cwd = getcwd()
nav_root = op.abspath(op.join(cwd, op.pardir))
data_folder = op.join(nav_root, 'vanilla', 'tests', 'data')
f = op.join(data_folder, id_qpcr_plate + '.xls')

qpcr_data = QpcrDataFile(f)
id_qpcr_inst_plate = qpcr_data.get_data_by_well()

C:\Users\Prasanna.Gnanaraj\AppData\Local\Programs\Python\Python36\lib\site-packages\pandas\core\ops.py:816: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


In [5]:
from hardware.labchip import LabChip

lc_plate = '20180103_B'
fds = glob(op.join(data_folder, '*' + lc_plate + '*.csv'))

files = {}
for f in fds:
    if 'PeakTable' in f:
        files['peak_file'] = f
    elif 'RawTable' in f:
        files['raw_file'] = f
    elif 'WellTable' in f:
        
        files['well_file'] = f
lc = LabChip(**files)
lc_inst_plate = lc.get_data_by_well()

In [6]:
from clients.expt_recipes.db_query import get_assay_amplicon_length
from clients.expt_recipes.vanilla.model_builders import build_qpcr_constituents
from clients.expt_recipes.lost import build_labchip_datas_from_inst_data
from clients.expt_recipes.vanilla.constituents import IdConstituents
from clients.expt_recipes.vanilla.model_builders import (build_id_qpcr_datas_from_inst_data,
                                                         get_wells_by_id_assay)
from clients.expt_recipes.db_query import create_qpcr_lc_mapping, get_lc_dilutions
from clients.expt_recipes.vanilla.models import VanillaMasterTable

In [7]:
%%time
id_plate_constituents = build_qpcr_constituents(expt_plates[id_qpcr_plate])

wells_by_id_assay = get_wells_by_id_assay(id_plate_constituents)

id_qpcr_datas = build_id_qpcr_datas_from_inst_data(id_plate_constituents, id_qpcr_inst_plate)

Wall time: 5.99 ms


In [8]:
%%time

assays = {}
for assay in wells_by_id_assay:
    for a in assay:
        assays[a] = int(get_assay_amplicon_length(a))

mapping = create_qpcr_lc_mapping(expt_plates[lc_plate])
dilutions = get_lc_dilutions(expt_plates[lc_plate])
lc_datas = build_labchip_datas_from_inst_data(
    id_plate_constituents, lc_inst_plate, mapping, assays, dilutions)

Wall time: 1.64 s


In [9]:
%%time
qwells = wells_by_id_assay[('Kp_KPC_5.x_KPC19_KPC24',)]
lcwells = []
for qw in qwells:
    if qw in mapping:
        lcwells.append(mapping[qw])
    else:
        lcwells.append(None)

nmt = VanillaMasterTable.create_from_models(qwells,
                                           id_qpcr_plate,
                                           lcwells,
                                           lc_plate,
                                           id_plate_constituents,
                                           id_qpcr_datas,
                                           lc_datas)

Wall time: 0 ns


In [10]:
nmt.rows[0]

VanillaMasterTableRow([('qPCR Plate', 'A81_E217_1'),
                       ('qPCR Well', 'A09'),
                       ('LC Plate', '20180103_B'),
                       ('LC Well', 'A09'),
                       ('ID Assay Name', ('Kp_KPC_5.x_KPC19_KPC24',)),
                       ('ID Assay Conc.', (0.2,)),
                       ('ID Template Name', ()),
                       ('ID Template Conc.', ()),
                       ('ID Human Name', ()),
                       ('ID Human Conc.', ()),
                       ('Ct', nan),
                       ('∆NTC Ct', nan),
                       ('Ct Call', False),
                       ('Tm1', 60.00170135498047),
                       ('Tm2', nan),
                       ('Tm3', nan),
                       ('Tm4', nan),
                       ('Tm Specif', False),
                       ('Tm NS', False),
                       ('Tm PD', True),
                       ('Specif ng/ul', 0),
                       ('NS ng/ul', 0),
  

In [11]:
import pandas as pd
df = pd.DataFrame(nmt.rows)
df.loc[:, ['qPCR Well', 'Ct', '∆NTC Ct', 'Ct Call', 'Tm Specif','Tm NS', 'Tm PD']]

,qPCR Well,Ct,∆NTC Ct,Ct Call,Tm Specif,Tm NS,Tm PD
0,A09,NaN,NaN,False,False,False,True
1,G09,NaN,NaN,False,False,True,False
2,H09,NaN,NaN,False,False,True,False
3,A10,NaN,NaN,False,False,False,True
4,G10,NaN,NaN,False,False,True,False
5,H10,NaN,NaN,False,False,True,True
6,B09,32.185207,7.814793,True,True,False,False
7,C09,32.877289,7.122711,True,True,False,False
8,D09,27.445021,12.554979,True,True,False,False
9,E09,23.967501,16.032499,True,True,False,False


In [12]:
grps = {}

for r in nmt.rows:
    grp_key = (r['ID Template Name'], r['ID Template Conc.'],
               r['ID Human Name'], r['ID Human Conc.'])
    grps.setdefault(grp_key, []).append(r)
grps.keys()

dict_keys([((), (), (), ()), ((), (), ('HgDNA_Promega287320',), (50.0,)), ((), (), ('HgDNA_Promega287320',), (500.0,)), (('K.pneumoniae-KPC_1705',), (10.0,), (), ()), (('K.pneumoniae-KPC_1705',), (100.0,), (), ()), (('K.pneumoniae-KPC_1705',), (1000.0,), (), ()), (('K.pneumoniae-KPC_1705',), (10000.0,), (), ())])

In [15]:
from clients.expt_recipes.vanilla.models import VanillaSummaryRow

In [16]:
VanillaSummaryRow.create_from_master_table_rows(
    grps[(('K.pneumoniae-KPC_1705',), (100.0,), (), ())])

VanillaSummaryRow([('qPCR Plate', 'A81_E217_1'),
                   ('LC Plate', '20180103_B'),
                   ('ID Assay Name', ('Kp_KPC_5.x_KPC19_KPC24',)),
                   ('ID Assay Conc.', (0.2,)),
                   ('ID Template Name', ('K.pneumoniae-KPC_1705',)),
                   ('ID Template Conc.', (100.0,)),
                   ('ID Human Name', ()),
                   ('ID Human Conc.', ()),
                   ('Reps', 2),
                   ('#Ct Pos', 2),
                   ('#Tm Specif', 2),
                   ('#Tm NS', 0),
                   ('#Tm PD', 0),
                   ('Min Ct', 27.3691349029541),
                   ('Mean Ct', 27.40707778930664),
                   ('Max Ct', 27.44502067565918),
                   ('Mean ∆NTC Ct', 12.59292221069336),
                   ('Min Tm1', 91.69805908203125),
                   ('Mean Tm1', 91.69805908203125),
                   ('Max Tm1', 91.69805908203125),
                   ('Mean Specif ng/ul', 34.1196482

In [14]:
srows = []
for g in grps:
    srows.append(VanillaSummaryRow.create_from_master_table_rows(grps[g]))

pd.DataFrame(srows)

,qPCR Plate,LC Plate,ID Assay Name,ID Assay Conc.,ID Template Name,ID Template Conc.,ID Human Name,ID Human Conc.,Reps,#Ct Pos,...,Min Ct,Mean Ct,Max Ct,Mean ∆NTC Ct,Min Tm1,Mean Tm1,Max Tm1,Mean Specif ng/ul,Mean NS ng/ul,Mean PD ng/ul
0,A81_E217_1,20180103_B,"(Kp_KPC_5.x_KPC19_KPC24,)","(0.2,)",(),(),(),(),2,0,...,NaN,NaN,NaN,NaN,60.001701,60.001701,60.001701,0.000000,0.0,0.0
1,A81_E217_1,20180103_B,"(Kp_KPC_5.x_KPC19_KPC24,)","(0.2,)",(),(),"(HgDNA_Promega287320,)","(50.0,)",2,0,...,NaN,NaN,NaN,NaN,80.122002,81.706818,83.291634,0.000000,0.0,0.0
2,A81_E217_1,20180103_B,"(Kp_KPC_5.x_KPC19_KPC24,)","(0.2,)",(),(),"(HgDNA_Promega287320,)","(500.0,)",2,0,...,NaN,NaN,NaN,NaN,89.079666,89.148571,89.217476,0.000000,0.0,0.0
3,A81_E217_1,20180103_B,"(Kp_KPC_5.x_KPC19_KPC24,)","(0.2,)","(K.pneumoniae-KPC_1705,)","(10.0,)",(),(),4,4,...,31.993595,32.513795,32.999088,7.486205,91.835876,91.904778,91.973679,16.182392,0.0,0.0
4,A81_E217_1,20180103_B,"(Kp_KPC_5.x_KPC19_KPC24,)","(0.2,)","(K.pneumoniae-KPC_1705,)","(100.0,)",(),(),2,2,...,27.369135,27.407078,27.445021,12.592922,91.698059,91.698059,91.698059,34.119648,0.0,0.0
5,A81_E217_1,20180103_B,"(Kp_KPC_5.x_KPC19_KPC24,)","(0.2,)","(K.pneumoniae-KPC_1705,)","(1000.0,)",(),(),2,2,...,23.967501,24.030610,24.093719,15.969390,91.698059,91.698059,91.698059,43.758225,0.0,0.0
6,A81_E217_1,20180103_B,"(Kp_KPC_5.x_KPC19_KPC24,)","(0.2,)","(K.pneumoniae-KPC_1705,)","(10000.0,)",(),(),2,2,...,20.041382,20.261299,20.481216,19.738701,91.698059,91.698059,91.698059,35.124069,0.0,0.0
